# 📊 ETF Portfolio Database Builder

Ce notebook construit une base de données complète d'ETF avec toutes les features nécessaires pour la sélection de portefeuille.

**Features calculées:**
- Rendement annualisé (CAGR) sur 1, 3, 5, 10 ans
- Volatilité annualisée
- Sharpe Ratio
- Max Drawdown
- Matrice de corrélation

## 1. Installation et imports

In [1]:
# Installation
!pip install pandas numpy yfinance openpyxl tqdm

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

## 2. Configuration

In [4]:
INPUT_FILE = "dataset/data_to_csv/etf_complete_data.xlsx"      # Ton fichier source
OUTPUT_FILE = "etf_portfolio_db.xlsx" # Fichier de sortie

YEARS_HISTORY = 10                     # Années d'historique à récupérer
RISK_FREE_RATE = 0.025                 # Taux sans risque (2.5%)

## 3. Chargement du fichier source

In [5]:
df_source = pd.read_excel(INPUT_FILE)

tickers = df_source['Ticker'].dropna().unique().tolist()

df_source.head()

,Ticker,Name,Description,% of Assets in Top 10,% of Assets in Top 15,% of Assets in Top 50,0.0%,0.15%,0.22%,0.35%,...,iShares US Transportation ETF,iShares iBonds 2026 Term High Yield and Income ETF,iShares iBonds Dec 2026 Term Corporate ETF,iShares iBonds Dec 2027 Term Corporate ETF,iShares iBonds Dec 2028 Term Corporate ETF,iShares iBonds Dec 2029 Term Corporate ETF,iShares iBonds Dec 2029 Term Treasury ETF,© 2025,Åland Islands,×
0,IEV,IEViShares Europe ETF,"Explore IEV for FREE on ETF Database: Price, H...",19.97%,26.09%,53.17%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Market data,NaN,Close
1,KXI,KXIiShares Global Consumer Staples ETF,"Explore KXI for FREE on ETF Database: Price, H...",50.91%,60.35%,88.82%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Market data,NaN,Close
2,NLR,NLRVanEck Uranium and Nuclear ETF,"Explore NLR for FREE on ETF Database: Price, H...",55.20%,75.29%,100.00%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Market data,NaN,Close
3,IXC,IXCiShares Global Energy ETF,"Explore IXC for FREE on ETF Database: Price, H...",59.86%,69.72%,99.79%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Market data,NaN,Close
4,TAN,TANInvesco Solar ETF,"Explore TAN for FREE on ETF Database: Price, H...",59.39%,75.56%,100.00%,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Market data,NaN,Close


## 4. Téléchargement des prix historiques (yfinance)

In [6]:
# 10ans de données
end_date = datetime.now()
start_date = end_date.replace(year=end_date.year - YEARS_HISTORY)

prices = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
prices = prices.dropna(how='all')  # Supprimer les lignes où tous les prix sont NaN

min_days = int(252 * YEARS_HISTORY * 0.99)
prices_10y = prices.dropna(axis=1, thresh=min_days)

print(prices_10y.shape)

[*********************100%***********************]  386 of 386 completed


(2515, 227)


In [7]:
prices_clean = prices_10y.dropna(axis=1, how="all")
prices_clean.head()
prices_clean.shape

(2515, 227)

# Téléchargement des volumes historiques

In [38]:
# Download volume data for tickers with 10Y prices
volumes = yf.download(
    tickers=prices_10y.columns.tolist(),
    start=start_date,
    end=end_date
)['Volume']

# Drop columns where volume is all NaN (just like prices)
volumes_10y = volumes.dropna(axis=1, thresh=min_days)

# Average daily volume over last year
TRADING_DAYS_PER_YEAR = 252
avg_daily_volume = volumes_10y.tail(TRADING_DAYS_PER_YEAR).mean()
df_liquidity = avg_daily_volume.reset_index()
df_liquidity.columns = ['Ticker', 'Avg_Daily_Volume']

[*********************100%***********************]  227 of 227 completed


## 5. Calcul des rendements journaliers

In [8]:
# Rendements journaliers
returns = prices_10y.pct_change().dropna()
returns

Ticker,AGGY,AIRR,AOA,AOK,AOM,AOR,ARKQ,ARKW,BOND,CEMB,...,WTV,XAR,XCEM,XMHQ,XMMO,XNTK,XSD,XSOE,XT,ZROZ
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-05,0.000000,-0.000619,0.001354,0.000953,0.000000,0.000780,0.004255,0.006688,0.001826,-0.010927,...,0.002565,0.008424,0.000000,0.001390,0.004452,-0.001898,-0.009743,0.000000,0.003163,-0.008132
2016-01-06,0.004591,-0.009908,-0.013069,-0.001905,-0.004721,-0.007533,-0.019597,-0.011515,0.002782,-0.003900,...,-0.013433,-0.009873,0.000000,-0.022216,-0.008524,-0.013501,-0.026001,-0.036221,-0.015970,0.020542
2016-01-07,0.000000,-0.038149,-0.017808,-0.007634,-0.008598,-0.013086,-0.017288,-0.037634,-0.002201,0.006089,...,-0.021073,-0.024928,0.000000,-0.020733,-0.034388,-0.026600,-0.032708,-0.018283,-0.023304,0.000442
2016-01-08,0.000000,0.000260,-0.009065,-0.002244,-0.004785,-0.007955,-0.034085,-0.010242,0.001151,-0.002161,...,-0.013413,-0.014749,0.000000,-0.006960,-0.011752,-0.013663,-0.016410,-0.020693,-0.012356,0.005295
2016-01-11,0.000691,-0.015861,0.000469,-0.002248,0.000000,0.000267,-0.029539,-0.022577,-0.001628,0.000000,...,0.004364,0.003593,0.000000,-0.017815,-0.012613,0.006826,0.006067,0.000000,-0.006687,-0.018346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-26,-0.001155,-0.003346,-0.000443,0.002978,0.000417,0.000459,-0.017403,-0.002813,0.000000,-0.000436,...,0.000988,-0.016444,0.006607,-0.000764,-0.000565,0.000035,-0.006420,0.008678,0.001230,-0.006632
2025-12-29,0.000907,-0.007109,-0.000443,-0.000247,-0.001042,-0.001375,-0.008540,-0.009295,0.001282,-0.000872,...,-0.001910,-0.006371,0.004201,-0.004489,-0.006854,-0.004105,-0.007284,-0.003574,-0.006229,0.003642
2025-12-30,0.000226,-0.009845,-0.001109,0.000247,-0.000835,-0.000459,-0.005168,-0.007585,-0.000107,0.001527,...,-0.000638,-0.006575,0.003399,-0.005373,-0.005549,-0.002701,-0.001658,0.002818,-0.000569,-0.004385


## 6. Calcul des métriques de performance

In [43]:
def calculate_metrics(prices_10y, returns, ticker):
    """
    Calcule toutes les métriques pour un ETF.
    """
    p = prices_10y[ticker].dropna()
    r = returns[ticker].dropna()

    if len(p) < 252:  # Minimum 1 an de données
        return None

    metrics = {'Ticker': ticker}

    # === RENDEMENTS ANNUALISÉS (CAGR) ===
    periods = {
        'Return_1Y': 252,
        'Return_3Y': 252 * 3,
        'Return_5Y': 252 * 5,
        'Return_10Y': 252 * 10
    }

    for name, days in periods.items():
        if len(p) >= 252:  # Minimum 1 year
            # Use the first available price in the period or the earliest if not enough
            start_idx = max(-days, -len(p))
            start_price = p.iloc[start_idx]
            end_price = p.iloc[-1]
            years = len(p.iloc[start_idx:]) / 252  # approximate years available
            cagr = (end_price / start_price) ** (1 / years) - 1
            metrics[name] = round(cagr * 100, 2)
        else:
            metrics[name] = np.nan

    # === VOLATILITÉ ANNUALISÉE ===
    metrics['Volatility_1Y'] = round(r.tail(252).std() * np.sqrt(252) * 100, 2)
    metrics['Volatility_10Y'] = round(r.std() * np.sqrt(252) * 100, 2)

    # === SHARPE RATIO ===
    annual_return = r.mean() * 252
    annual_vol = r.std() * np.sqrt(252)
    if annual_vol > 0:
        metrics['Sharpe_Ratio'] = round((annual_return - RISK_FREE_RATE) / annual_vol, 2)
    else:
        metrics['Sharpe_Ratio'] = np.nan

    # === MAX DRAWDOWN ===
    cummax = p.cummax()
    drawdown = (p - cummax) / cummax
    metrics['Max_Drawdown'] = round(drawdown.min() * 100, 2)

    # === SORTINO RATIO ===
    downside_returns = r[r < 0]
    if len(downside_returns) > 0 and downside_returns.std() > 0:
        downside_vol = downside_returns.std() * np.sqrt(252)
        metrics['Sortino_Ratio'] = round((annual_return - RISK_FREE_RATE) / downside_vol, 2)
    else:
        metrics['Sortino_Ratio'] = np.nan
    
     # === Quality Score ===
    # Base: 45% Return_10Y + 45% Sharpe_Ratio
    quality_score = 0.45 * metrics['Return_10Y'] + 0.45 * metrics['Sharpe_Ratio']

    # Add liquidity if available
    if df_liquidity is not None:
        vol_row = df_liquidity.loc[df_liquidity['Ticker'] == ticker]
        if not vol_row.empty:
            avg_vol_log = np.log1p(vol_row['Avg_Daily_Volume'].values[0])
            quality_score += 0.1 * avg_vol_log  # 10% weight for liquidity

    metrics['Quality_Score'] = round(quality_score, 2)

    return metrics

In [44]:
# Calculer les métriques pour tous les ETF
all_metrics = []

for ticker in tqdm(prices_10y.columns, desc="Calcul des métriques"):
    m = calculate_metrics(prices_10y, returns, ticker)
    if m:
        all_metrics.append(m)

df_metrics = pd.DataFrame(all_metrics)
df_metrics.head(10)

Calcul des métriques: 100%|██████████| 227/227 [00:00<00:00, 896.73it/s] 


,Ticker,Return_1Y,Return_3Y,Return_5Y,Return_10Y,Volatility_1Y,Volatility_10Y,Sharpe_Ratio,Max_Drawdown,Sortino_Ratio,Quality_Score
0,AGGY,7.26,5.42,-0.42,2.24,5.16,5.47,-0.03,-20.97,-0.03,2.15
1,AIRR,32.08,33.10,24.61,20.46,27.41,26.01,0.75,-42.37,1.05,10.87
2,AOA,20.24,17.62,9.42,10.03,13.46,13.56,0.59,-28.38,0.71,5.93
3,AOK,11.79,9.74,3.58,5.19,6.56,6.65,0.42,-18.93,0.51,3.71
4,AOM,13.57,11.34,4.78,6.17,8.01,7.89,0.48,-19.96,0.58,4.18
5,AOR,16.95,14.47,7.11,8.12,10.39,10.65,0.55,-22.95,0.68,5.15
6,ARKQ,53.71,44.35,9.73,21.01,36.56,29.43,0.71,-59.89,1.02,10.99
7,ARKW,40.23,60.74,1.30,23.35,38.90,37.47,0.68,-80.01,0.92,12.05
8,BOND,8.33,5.85,0.15,2.44,4.97,5.06,0.01,-19.71,0.01,2.38
9,CEMB,8.77,7.84,1.72,4.32,4.06,6.49,0.30,-20.84,0.33,3.15


# 7. Selection des ETF les moins corrélés avec les quality scores les plus élevés et return > 10% 

In [110]:
returns_clean = returns[prices_10y.columns]
correlation_matrix = returns_clean.corr().abs()

upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
threshold = 0.8
to_keep = []
processed = set()
top_n = 2  # keep top 2 per correlated cluster

for col in corr_matrix.columns:
    if col in processed:
        continue

    # Find ETFs correlated above threshold
    correlated = upper.index[upper[col] > threshold].tolist()
    correlated.append(col)

    # Keep only ETFs that exist in returns_clean
    correlated = [t for t in correlated if t in returns_clean.columns]

    if correlated:
        # Sort by Quality_Score if available, else by Return_10Y
        if 'Quality_Score' in df_final.columns:
            top_etfs = df_final[df_final['Ticker'].isin(correlated)] \
                .sort_values('Quality_Score', ascending=False)['Ticker'].tolist()
        else:
            top_etfs = df_final[df_final['Ticker'].isin(correlated)] \
                .sort_values('Return_10Y', ascending=False)['Ticker'].tolist()

        # Keep top_n
        to_keep.extend(top_etfs[:top_n])

    processed.update(correlated)

# Filter metrics dataframe for the selected ETFs
selected_etfs = df_final[
    (df_final['Ticker'].isin(to_keep)) &
    (df_final['Return_10Y'] > 10)
].copy()

# Optional: sort by Quality_Score or Return_10Y
selected_etfs = selected_etfs.sort_values(
    'Quality_Score' if 'Quality_Score' in df_final.columns else 'Return_10Y',
    ascending=False
)

print(f"✅ Final ETFs after correlation and Return_10Y filter: {len(selected_etfs)}")
selected_etfs.head(20)

✅ Final ETFs after correlation and Return_10Y filter: 12


,Ticker,Name,Expense Ratio,Annual Dividend Yield,Asset Class,Asset Class Size,Asset Class Style,Region (General),Region (Specific),Segment,...,Return_3Y,Return_5Y,Return_10Y,Volatility_1Y,Volatility_10Y,Sharpe_Ratio,Max_Drawdown,Sortino_Ratio,Quality_Score,Meets_Return_Criteria
70,PSI,Invesco Semiconductors ETF,0.56,0.09,Equity,Multi-Cap,Growth,North America,U.S.,Equity: U.S. Semiconductors,...,37.05,19.53,26.53,43.40,34.23,0.79,-44.85,1.07,13.40,True
54,IYW,iShares U.S. Technology ETF,0.38,0.14,Equity,Large-Cap,Growth,North America,U.S.,Equity: U.S. Information Technology,...,40.74,18.85,23.29,27.78,24.96,0.86,-39.44,1.11,12.24,True
39,RING,iShares MSCI Global Gold Miners ETF,0.39,0.84,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Gold Miners,...,50.85,22.02,22.28,38.78,36.61,0.66,-52.04,0.98,11.62,True
42,PICK,iShares MSCI Global Metals & Mining Producers ETF,0.39,2.89,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Metals & Mining,...,12.29,12.41,16.80,24.94,28.74,0.60,-52.73,0.83,9.10,True
2,NLR,VanEck Uranium and Nuclear ETF,0.56,2.51,Equity,Multi-Cap,Value,Developed Markets,Broad,Equity: Global Nuclear Energy,...,38.26,25.12,14.79,40.57,22.31,0.62,-34.35,0.81,8.24,True
82,UTES,Virtus Reaves Utilities ETF,0.49,1.41,Equity,Multi-Cap,Blend,North America,U.S.,Equity: U.S. Utilities,...,21.93,17.66,14.47,24.69,19.76,0.66,-35.39,0.80,8.01,True
41,LIT,Global X Lithium & Battery Tech ETF,0.75,0.41,Equity,Micro-Cap,Blend,Developed Markets,Broad,Equity: Global Mobility,...,5.62,2.53,14.45,31.87,30.34,0.51,-65.91,0.74,7.99,True
44,NANR,SPDR S&P North American Natural Resources ETF,0.35,1.77,Equity,Large-Cap,Blend,North America,Broad,Equity: North America Natural Resources,...,11.11,18.87,13.58,21.27,24.02,0.55,-49.15,0.71,7.43,True
36,GUNR,FlexShares Morningstar Global Upstream Natural...,0.46,3.51,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Natural Resources,...,5.92,11.36,11.22,16.65,20.83,0.50,-43.04,0.62,6.59,True
101,XCEM,Columbia EM Core ex-China ETF,0.16,NaN,Equity,Multi-Cap,Blend,Emerging Markets,Broad,Equity: Emerging Markets Ex-China - Total Market,...,18.33,8.29,10.94,16.97,19.44,0.50,-40.92,0.64,6.36,True


## 8. Fusion avec les données source

In [93]:
# Colonnes utiles du fichier source
source_cols = [
    'Ticker', 'Name',
    'Expense Ratio', 'Annual Dividend Yield',
    'Asset Class', 'Asset Class Size', 'Asset Class Style',
    'Region (General)', 'Region (Specific)',
    'Segment', 'Beta',
    'Number of Holdings', 'Inception',
    'Issuer', 'Index Tracked'
]

# Garder seulement les colonnes qui existent
existing_cols = [c for c in source_cols if c in df_source.columns]
df_source_clean = df_source[existing_cols].copy()

# Nettoyer les pourcentages
pct_cols = ['Expense Ratio', 'Annual Dividend Yield']
for col in pct_cols:
    if col in df_source_clean.columns:
        df_source_clean[col] = df_source_clean[col].astype(str).str.replace('%', '')
        df_source_clean[col] = pd.to_numeric(df_source_clean[col], errors='coerce')

# Nettoyer le nom
if 'Name' in df_source_clean.columns:
    df_source_clean['Name'] = df_source_clean.apply(
        lambda row: row['Name'][len(row['Ticker']):] if str(row['Name']).startswith(str(row['Ticker'])) else row['Name'],
        axis=1
    )

print(f"✅ Données source nettoyées: {len(existing_cols)} colonnes")

✅ Données source nettoyées: 15 colonnes


In [105]:
df_final = pd.merge(df_source_clean, df_metrics, on='Ticker', how='inner')
df_final['Meets_Return_Criteria'] = df_final['Return_10Y'] >= 10

print(f"✅ ETFs with Return_10Y >= 10%: {len(selected_etfs)}")

✅ ETFs with Return_10Y >= 10%: 12


In [104]:
# Merge source info
selected_etfs = selected_etfs.merge(df_source_clean, on='Ticker', how='left')

# Merge liquidity info
selected_etfs = selected_etfs.merge(df_liquidity, on='Ticker', how='left')

print(f"✅ Final ETFs after merging all data: {len(selected_etfs)}")
selected_etfs.head(20)

✅ Final ETFs after merging all data: 12


,Ticker,Name_x,Expense Ratio_x,Annual Dividend Yield_x,Asset Class_x,Asset Class Size_x,Asset Class Style_x,Region (General)_x,Region (Specific)_x,Segment_x,...,Asset Class Style,Region (General),Region (Specific),Segment,Beta,Number of Holdings,Inception,Issuer,Index Tracked,Avg_Daily_Volume_y
0,PSI,Invesco Semiconductors ETF,0.56,0.09,Equity,Multi-Cap,Growth,North America,U.S.,Equity: U.S. Semiconductors,...,Growth,North America,U.S.,Equity: U.S. Semiconductors,1.57,31.0,"Jun 23, 2005",Invesco,Dynamic Semiconductors Intellidex Index,60858.730159
1,IYW,iShares U.S. Technology ETF,0.38,0.14,Equity,Large-Cap,Growth,North America,U.S.,Equity: U.S. Information Technology,...,Growth,North America,U.S.,Equity: U.S. Information Technology,1.23,143.0,"May 15, 2000","BlackRock, Inc.",Dow Jones U.S. Technology Index,923192.460317
2,RING,iShares MSCI Global Gold Miners ETF,0.39,0.84,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Gold Miners,...,Blend,Developed Markets,Broad,Equity: Global Gold Miners,0.50,43.0,"Jan 31, 2012","BlackRock, Inc.",MSCI ACWI Select Gold Miners Investable Market...,416807.539683
3,PICK,iShares MSCI Global Metals & Mining Producers ETF,0.39,2.89,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Metals & Mining,...,Blend,Developed Markets,Broad,Equity: Global Metals & Mining,1.09,230.0,"Jan 31, 2012","BlackRock, Inc.",MSCI ACWI Select Metals & Mining Producers Ex ...,313818.253968
4,NLR,VanEck Uranium and Nuclear ETF,0.56,2.51,Equity,Multi-Cap,Value,Developed Markets,Broad,Equity: Global Nuclear Energy,...,Value,Developed Markets,Broad,Equity: Global Nuclear Energy,0.75,27.0,"Aug 13, 2007",VanEck,MVIS Global Uranium & Nuclear Energy,460709.920635
5,UTES,Virtus Reaves Utilities ETF,0.49,1.41,Equity,Multi-Cap,Blend,North America,U.S.,Equity: U.S. Utilities,...,Blend,North America,U.S.,Equity: U.S. Utilities,0.73,20.0,"Sep 23, 2015",Virtus Investment Partners,ACTIVE - No Index,172829.365079
6,LIT,Global X Lithium & Battery Tech ETF,0.75,0.41,Equity,Micro-Cap,Blend,Developed Markets,Broad,Equity: Global Mobility,...,Blend,Developed Markets,Broad,Equity: Global Mobility,1.05,41.0,"Jul 22, 2010","Mirae Asset Global Investments Co., Ltd.",Stuttgart Solactive AG Global Lithium (USD),299282.142857
7,NANR,SPDR S&P North American Natural Resources ETF,0.35,1.77,Equity,Large-Cap,Blend,North America,Broad,Equity: North America Natural Resources,...,Blend,North America,Broad,Equity: North America Natural Resources,0.75,176.0,"Dec 15, 2015",State Street,S&P BMI North American Natural Resources Index,43401.984127
8,GUNR,FlexShares Morningstar Global Upstream Natural...,0.46,3.51,Equity,Large-Cap,Blend,Developed Markets,Broad,Equity: Global Natural Resources,...,Blend,Developed Markets,Broad,Equity: Global Natural Resources,0.72,120.0,"Sep 16, 2011",Northern Trust Corp.,Morningstar Global Upstream Natural Resources ...,519331.349206
9,XCEM,Columbia EM Core ex-China ETF,0.16,NaN,Equity,Multi-Cap,Blend,Emerging Markets,Broad,Equity: Emerging Markets Ex-China - Total Market,...,Blend,Emerging Markets,Broad,Equity: Emerging Markets Ex-China - Total Market,0.84,329.0,"Sep 02, 2015",Ameriprise Financial,MSCI Emerging Markets ex China Index,191353.968254


In [106]:
# Aperçu de la base finale
display_cols = ['Ticker', 'Name', 'Return_10Y', 'Volatility_10Y', 'Sharpe_Ratio', 'Max_Drawdown', 'Meets_Return_Criteria']
df_final[display_cols].sort_values('Return_10Y', ascending=False).head(15)

,Ticker,Name,Return_10Y,Volatility_10Y,Sharpe_Ratio,Max_Drawdown,Meets_Return_Criteria
70,PSI,Invesco Semiconductors ETF,26.53,34.23,0.79,-44.85,True
66,XSD,State Street SPDR S&P Semiconductor ETF,23.42,34.33,0.71,-42.27,True
65,ARKW,ARK Next Generation Internet ETF,23.35,37.47,0.68,-80.01,True
54,IYW,iShares U.S. Technology ETF,23.29,24.96,0.86,-39.44,True
53,VGT,Vanguard Information Technology ETF,22.84,24.44,0.86,-35.07,True
55,FTEC,Fidelity MSCI Information Technology Index ETF,22.61,24.53,0.85,-34.95,True
57,IGM,iShares Expanded Tech Sector ETF,22.40,24.38,0.85,-40.68,True
39,RING,iShares MSCI Global Gold Miners ETF,22.28,36.61,0.66,-52.04,True
67,XNTK,State Street SPDR NYSE Technology ETF,22.11,26.42,0.79,-48.28,True
58,IXN,iShares Global Tech ETF,21.89,24.07,0.84,-36.30,True


## 9. Export vers Excel

In [109]:
# Créer le fichier Excel avec plusieurs onglets
with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:

    # Onglet 1: Toutes les données
    df_final.to_excel(writer, sheet_name='ETF_Data', index=False)

    # Onglet 2: Matrice de corrélation
    correlation_matrix.to_excel(writer, sheet_name='Correlation_Matrix')

    # Onglet 3: ETF qualifiés (rendement > 10%)
    selected_etfs.to_excel(writer, sheet_name='Qualified_ETFs', index=False)

    # Onglet 4: Prix historiques
    prices.to_excel(writer, sheet_name='Price_History')

print(f"Fichier exporté: {OUTPUT_FILE}")

Fichier exporté: etf_portfolio_db.xlsx


Abdelhamid Test